# Function Calling

Function calling is a powerful feature of OpenAI `gpt-4` and other models. It essentially is a **Function Picker and Parameter Provider**. It can help you choose which function, if any, to invoke with which parameters, under provided context and instruction. 

LionAGI allows simple usage of function callings in the `Session` object. 

In [1]:
tool_schema = {
    "type": "function",
    "function": {
        "name": "multiply",
        "description": "Perform multiplication on two numbers",
        "parameters": {
            "type": "object",
            "properties": {
                "number1": {
                    "type": "number",
                    "description": "a number to multiply, e.g. 5.34",
                },
                "number2": {
                    "type": "number",
                    "description": "a number to multiply, e.g. 17",
                },
            },
            # specify which parameters are required for the model to respond 
            # when function calling
            "required": ["number1", "number2"],
        },
    }
}

def multiply(number1, number2):
    return number1*number2

In [2]:
from lionagi.core.schema import Tool

# created a tool object
tools = Tool(func=multiply, schema_=tool_schema)

In [3]:
# add some messages for our session
system = """
    you are asked to perform as a function picker and parameter provider
"""

task = """
    Think step by step, understand the following basic math question and 
    provide parameters for function calling.
"""

# json format for the task
json_format = {"number1": "x", "number2": "y"}
instruct1 = {"Task": task, "json_format": json_format}

In [4]:
from lionagi import Session

In [5]:
session = Session(system=system, tools=tools)

In [6]:


# now we provide the speific problems as contexts 
question1 = """
    There are [basketball, football, backpack, water bottle, strawberry, 
    tennis ball, rockets]. each comes in four different colors, what is 
    the number of unique kinds of ball?
"""

question2 = """
    There are three fruits in total, each with 2 different colors, how 
    many unique kinds of fruits are there?
"""



response = await session.chat(
    instruct1, 
    context={"Question1": question1, "question2": question2}, 
    tools=True, 
    response_format={'type':"json_object"}
)

for function call to work, you either need to enforce 

- `response_format={'type':"json_object"}`, or `tool_choice = 'auto'` 

- if you use 'auto', you let model choose what to do, 

- if you want to force a function call, use `tool_choice={"type": "function", "function": {"name": "my_function"}}`

In [7]:
# responses from action requests are dictionary
import lionagi as li

print(li.to_readable_dict(response))

{
    "action_request": [
        {
            "recipient_name": "functions.multiply",
            "parameters": {
                "number1": 3,
                "number2": 2
            }
        }
    ]
}


In [8]:
li.lcall(response['action_request'], lambda x: print(x));

{'recipient_name': 'functions.multiply', 'parameters': {'number1': 3, 'number2': 2}}


In [9]:
# messages is a pandas dataframe
session.messages

,node_id,timestamp,role,sender,content
0,16e32bafe1edb0055735ebbc68b8b16a,2024-02-27T20_21_42_264068,system,system,"{""system_info"": ""\n you are asked to perfor..."
1,b82afd7589b0b1477d8ebd62e4005133,2024-02-27T20_21_42_268155,user,user,"{""instruction"": {""Task"": ""\n Think step by ..."
2,3a834019fe58136990313f120d598f90,2024-02-27T20_21_46_025691,assistant,action_request,"{""action_request"": [{""recipient_name"": ""functi..."
3,a727c54ef98774c7a64cf1f6fb0fe3a3,2024-02-27T20_21_46_028620,assistant,action_response,"{""action_response"": {""function"": ""multiply"", ""..."


In [10]:
# since the response is an output from action, you need to use the key `action_response`
# now let us check the function responses
df = session.default_branch.action_response

for content in df['content']:
    dict_ = li.to_dict(content)
    print(dict_)

{'action_response': {'function': 'multiply', 'arguments': {'number1': 3, 'number2': 2}, 'output': 6}}


In [11]:
log = list(session.default_branch.datalogger.log)[0]
log.input_data

{'messages': [{'role': 'system',
   'content': '{"system_info": "\\n    you are asked to perform as a function picker and parameter provider\\n"}'},
  {'role': 'user',
   'content': '{"instruction": {"Task": "\\n    Think step by step, understand the following basic math question and \\n    provide parameters for function calling.\\n", "json_format": {"number1": "x", "number2": "y"}}, "context": {"Question1": "\\n    There are [basketball, football, backpack, water bottle, strawberry, \\n    tennis ball, rockets]. each comes in four different colors, what is \\n    the number of unique kinds of ball?\\n", "question2": "\\n    There are three fruits in total, each with 2 different colors, how \\n    many unique kinds of fruits are there?\\n"}}'}],
 'model': 'gpt-4-turbo-preview',
 'frequency_penalty': 0,
 'n': 1,
 'presence_penalty': 0,
 'response_format': {'type': 'json_object'},
 'temperature': 0.7,
 'top_p': 1,
 'tools': [{'type': 'function',
   'function': {'name': 'multiply',
    '

In [12]:
session.default_branch.log_to_csv()

1 logs saved to data/logs/log_20240227202146.csv


In [13]:
session.default_branch.load_log('data/logs/log_20240227202146.csv')

Loaded 1 logs from data/logs/log_20240227202146.csv


In [14]:
log = list(session.default_branch.datalogger.log)[0]

In [15]:
log.input_data

{'messages': [{'role': 'system',
   'content': '{"system_info": "\\n    you are asked to perform as a function picker and parameter provider\\n"}'},
  {'role': 'user',
   'content': '{"instruction": {"Task": "\\n    Think step by step, understand the following basic math question and \\n    provide parameters for function calling.\\n", "json_format": {"number1": "x", "number2": "y"}}, "context": {"Question1": "\\n    There are [basketball, football, backpack, water bottle, strawberry, \\n    tennis ball, rockets]. each comes in four different colors, what is \\n    the number of unique kinds of ball?\\n", "question2": "\\n    There are three fruits in total, each with 2 different colors, how \\n    many unique kinds of fruits are there?\\n"}}'}],
 'model': 'gpt-4-turbo-preview',
 'frequency_penalty': 0,
 'n': 1,
 'presence_penalty': 0,
 'response_format': {'type': 'json_object'},
 'temperature': 0.7,
 'top_p': 1,
 'tools': [{'type': 'function',
   'function': {'name': 'multiply',
    '

In [16]:
a = log.input_data

In [17]:
a

{'messages': [{'role': 'system',
   'content': '{"system_info": "\\n    you are asked to perform as a function picker and parameter provider\\n"}'},
  {'role': 'user',
   'content': '{"instruction": {"Task": "\\n    Think step by step, understand the following basic math question and \\n    provide parameters for function calling.\\n", "json_format": {"number1": "x", "number2": "y"}}, "context": {"Question1": "\\n    There are [basketball, football, backpack, water bottle, strawberry, \\n    tennis ball, rockets]. each comes in four different colors, what is \\n    the number of unique kinds of ball?\\n", "question2": "\\n    There are three fruits in total, each with 2 different colors, how \\n    many unique kinds of fruits are there?\\n"}}'}],
 'model': 'gpt-4-turbo-preview',
 'frequency_penalty': 0,
 'n': 1,
 'presence_penalty': 0,
 'response_format': {'type': 'json_object'},
 'temperature': 0.7,
 'top_p': 1,
 'tools': [{'type': 'function',
   'function': {'name': 'multiply',
    '